# Rossmann Sales Forecasting — Time Series Modeling

---

## 1. Introduction

## Problem Statement

Rossmann operates over 3,000 drugstores across Europe. Due to the short shelf life of many pharmaceutical products, it's essential to forecast daily sales accurately.

Currently, store managers manually forecast daily sales for the next six weeks. To improve consistency and accuracy, we're tasked with building a **data-driven time-series model** to automate this process.

---

### Objective:

Build a robust **time-series forecasting pipeline** to predict **daily sales for the next 6 weeks**, for **9 key Rossmann stores** using:
- Time-series decomposition
- Stationarity checks (ADF test)
- Cointegration tests (Johansen)
- VAR or VARMAX modeling
- MAPE as evaluation metric


## 2. Load Data & Basic Inspection

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.api import VARMAX
from statsmodels.tsa.stattools import ccf
from statsmodels.tsa.seasonal import seasonal_decompose

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# set_option to see maximum data

pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("max_colwidth", None)

In [ ]:
# Load CSVs
train_df = pd.read_csv(r"C:\Users\lucky\Desktop\Capstone Project\Dataset\train.csv", parse_dates=["Date"])
store_df = pd.read_csv(r"C:\Users\lucky\Desktop\Capstone Project\Dataset\store.csv")

In [ ]:
# Checking data shapes
print("Train data shape:", train_df.shape)
print("Store data shape:", store_df.shape)

In [ ]:
# Samples of train data and Store data

print("-----------  Train dataset Sample  ----------")
print(train_df.head())
print("-"*100)
print("-----------  Store dataset Sample  ----------")
print(store_df.head())

In [ ]:
# Checking the datatype of the columns in the "Train" and "Store" dataset

print("----------  Train Data Types  ----------")
print(train_df.info())
print("-"*100)
print("----------  Store Data Types  ----------")
print(store_df.info())

In [ ]:
# Checking the summary of the "Train" and "Store" dataset

print("----------  Summary of Train Datasets  ----------")
print(train_df.describe())
print("-"*100)
print("----------  Summary of Store Datasets  ----------")
print(store_df.describe())

#### Dataset Cleaning and Dataset Transformation

In [ ]:
# Checking percentage of missing/null values

print("-----  Percentage of Missing/null values in Store Dataset  -----")
print(round(100*(store_df.isnull().sum()/len(store_df)),2))
print("-"*100)
print("-----  Percentage of Missing/null values in Train Dataset  -----")
print(round(100*(train_df.isnull().sum()/len(train_df)),2))

## 3. Dataset Cleaning and Transformation

In this step, we are going to clean and prepare the data by:

- Merging training and store metadata
- Focusing on a subset of selected stores
- Handling missing values appropriately
- Creating a new feature (`Promo2Active`) to reflect ongoing promotions
- Fixing logical inconsistencies in `CompetitionDistance`
- Sorting the data by `Store` and `Date` for time-series modeling

### **Load and Merge Datasets**

In [ ]:
# Merge training and store metadata

df = pd.merge(train_df, store_df, on='Store', how='left')

### **Filter for Selected Stores**

##### Focusing on 9 Key Stores for Modeling

The company has decided to focus only on 9 key stores across Europe. These stores have been selected based on their high revenue contributions and strategic importance to the business. 

By narrowing the scope to these specific stores, we can build more targeted and efficient forecasting models. Once the modeling approach is validated and performs well for these stores, it can be scaled to include the remaining stores in the future.

The selected stores for this case study are: **Store 1, 3, 8, 9, 13, 25, 29, 31, and 46**.


In [ ]:
# Focus on 9 selected stores for time-series modeling

selected_stores = [1, 3, 8, 9, 13, 25, 29, 31, 46]
df = df[df["Store"].isin(selected_stores)]

### **Drop Leakage Columns and Handle Missing Values**

In [ ]:
# Drop 'Customers' column — it's highly correlated with Sales and not known in advance

df.drop(columns=["Customers"], inplace=True)


# Fill NA values in specified competition/promo columns with 0

comp_cols = ["CompetitionDistance", "CompetitionOpenSinceMonth", "CompetitionOpenSinceYear", 
             "Promo2SinceWeek", "Promo2SinceYear", "PromoInterval"]

df[comp_cols] = df[comp_cols].fillna(0)

### **Fix Logical Inconsistencies**

In [ ]:
# If there is no competition, opening month/year should be zero as well

df.loc[df["CompetitionDistance"] == 0, "CompetitionOpenSinceMonth"] = 0
df.loc[df["CompetitionDistance"] == 0, "CompetitionOpenSinceYear"] = 0

### **Feature Engineering - Promo2 Active Flag**

In [ ]:
# Ensure Date is datetime type

df["Date"] = pd.to_datetime(df["Date"])


# Helper to determine if Promo2 is active in the current month

def is_promo2_active(row):
    if row["Promo2"] == 1 and row["PromoInterval"] != 0:
        promo_months = row["PromoInterval"].split(",")
        return row["Date"].strftime("%b") in promo_months
    return False

df["Promo2Active"] = df.apply(is_promo2_active, axis=1).astype(int)

### **Final Sort and Reset**

In [ ]:
# Sort by Store and Date for sequential modeling

df.sort_values(by=["Store", "Date"], inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
# Final shape and sample
print("Merged Dataset Shape:- ",df.shape)

In [ ]:
# Check cleaned sample dataset
print("-----------  Sample Merged Dataset  ---------- ")
print(df.head())

In [ ]:
# Summary of Datasets
print("----------  Summary of Merged Datasets  ----------")
print(df.describe())

In [ ]:
# Merged datatypes
print("----------  Information of Merged Datasets  ----------")
print(df.info())

In [ ]:
# Checking percentage of missing/null values
print("----------  Percentage of Missing/null Values of Merged Datasets ----------")
round(100*(df.isnull().sum()/len(df)),2)

## 4. Outlier Removal & Skewness Check

##### Outlier Capping and Skewness Check

Outliers, especially in the upper range of sales, can heavily skew time-series models. To mitigate this, we cap extreme values above the 99th percentile for each store.

In [ ]:
# Remove outliers above 99th percentile of Sales per store
for store in selected_stores:
    p99 = df[df.Store == store]["Sales"].quantile(0.99)
    df.loc[(df.Store == store) & (df["Sales"] > p99), "Sales"] = p99

In [ ]:
# Check skewness
sns.histplot(df["Sales"], kde=True)
plt.title("Sales Distribution after Outlier Removal")
plt.show()

Additionally, we observed many zero-sales days. Upon investigation, these are mostly days when the store was closed. Since no sales can occur when the store is closed, we remove those rows (where `Open = 0`) to avoid skewing the distribution.

In [ ]:
# Remove closed days (they contribute to many zero sales)
df = df[df["Open"] == 1]

In [ ]:
# Check skewness
sns.histplot(df["Sales"], kde=True)
plt.title("Sales Distribution after Outlier Removal")
plt.show()

The updated sales distribution plot now reflects realistic active sales, which helps in improving model accuracy and stability.

In [ ]:
# Detecting outliers at 90, 95 and 99 percentile

df.describe(percentiles=[0.90,0.95,0.99]).round(2)

- Outliers at the 99th percentile can skew the time series model drastically.
- We remove extreme sales values to ensure the stationarity test and variance structure remain valid.

## 5. Stratified Train-Test Split (Per Store)

In [ ]:
# Split last 6 weeks as test set (per store)
df["is_test"] = 0

for store in selected_stores:
    store_data = df[df.Store == store]
    cutoff_date = store_data["Date"].max() - pd.Timedelta(days=42)
    df.loc[(df.Store == store) & (df["Date"] > cutoff_date), "is_test"] = 1

# Confirm split
df["is_test"].value_counts()

- Since this is time series, a temporal split ensures the model is trained on past data and tested on future unseen periods.
- By maintaining a 6-week window for each store, we simulate real-world forecasting conditions.

## 6. Exploratory Data Analysis (EDA) Per Store

In [ ]:
# 📈 Plot Sales time series for each store
for store in selected_stores:
    plt.figure(figsize=(10, 4))
    store_data = df[df.Store == store]
    plt.plot(store_data["Date"], store_data["Sales"], label=f"Store {store}")
    plt.title(f" Sales over Time - Store {store}")
    plt.xlabel("Date")
    plt.ylabel("Sales")
    plt.legend()
    plt.tight_layout()
    plt.show()

Visualizing each store’s sales gives us:
- A sense of trend and seasonality
- Check for visible change points or anomalies
- Guide whether decomposition or transformation is needed

## 7. Seasonal Decomposition

In [ ]:
from statsmodels.tsa.seasonal import STL

for store in selected_stores:
    store_df = df[(df.Store == store) & (df.Open == 1)]
    store_ts = store_df.set_index("Date")["Sales"].asfreq("D").fillna(method="ffill")

    stl = STL(store_ts, seasonal=7)
    result = stl.fit()

    result.plot()
    plt.suptitle(f"STL Decomposition - Store {store}", fontsize=14)
    plt.tight_layout()
    plt.show()

- Helps visualize **trend**, **seasonality**, and **residuals**.
- Identifies whether deseasonalizing is needed before modeling.

## 8. Augmented Dickey-Fuller (ADF) Test

In [ ]:
def adf_test(series, title=''):
    """
    Perform ADF Test and print result.
    """
    print(f"\n ADF Test: {title}")
    result = adfuller(series.dropna(), autolag='AIC')
    labels = ['ADF Test Statistic', 'p-value', '#Lags Used', 'Num Observations Used']
    for value, label in zip(result, labels):
        print(f"{label} : {value}")
    if result[1] <= 0.05:
        print("Series is Stationary")
    else:
        print("Series is Non-Stationary")

# Apply ADF Test on raw sales per store
for store in selected_stores:
    print(f"\n📍 Store {store}")
    store_df = df[(df.Store == store) & (df.Open == 1)]
    store_sales = store_df.set_index("Date")["Sales"].asfreq("D").fillna(method="ffill")
    adf_test(store_sales, f"Store {store} - Sales")

- Checks for stationarity in the series.
- Stationarity is a key assumption in most time-series models like VAR/VARMAX.

## 9. Differencing (If Non-Stationary)

In [ ]:
# Apply differencing where needed and re-test
store_diff_dict = {}

for store in selected_stores:
    print(f"\n Store {store}")
    store_df = df[(df.Store == store) & (df.Open == 1)]
    store_sales = store_df.set_index("Date")["Sales"].asfreq("D").fillna(method="ffill")

    adf_result = adfuller(store_sales.dropna(), autolag='AIC')[1]
    if adf_result > 0.05:
        # First-order differencing
        store_sales_diff = store_sales.diff().dropna()
        print(" Applying First Differencing")
        adf_test(store_sales_diff, f"Store {store} - Sales (1st diff)")
        store_diff_dict[store] = store_sales_diff
    else:
        store_diff_dict[store] = store_sales

### Differencing

- Transforms a non-stationary series to stationary.
- VAR requires stationarity; this is a necessary step before modeling.

In [ ]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def johansen_test(df_subset, store):
    """
    Apply Johansen Test on Sales and Promo2Active series.
    Checks for cointegration to determine if the variables have a long-term equilibrium relationship.
    """
    df_subset = df_subset.set_index("Date")[["Sales", "Promo2Active"]].asfreq("D").fillna(method="ffill")

    # Skip if too few observations
    if len(df_subset) < 150:
        print(f"Skipping Store {store}: Not enough data points.")
        return

    # Skip if any column has no variation
    if df_subset["Sales"].nunique() < 2 or df_subset["Promo2Active"].nunique() < 2:
        print(f"Skipping Store {store}: One or more series has no variation.")
        return

    try:
        coint_result = coint_johansen(df_subset, det_order=0, k_ar_diff=1)
        print(f"\nJohansen Test - Store {store}")
        trace_stat = coint_result.lr1
        critical_values = coint_result.cvt[:, 1]  # 5% significance
        for i, (stat, crit) in enumerate(zip(trace_stat, critical_values)):
            print(f"Rank {i}: Trace Stat = {stat:.2f}, Crit Value (5%) = {crit}")
            if stat > crit:
                print("  ➤ Cointegration Exists at Rank", i)
            else:
                print("  ✖ No Cointegration at Rank", i)
    except Exception as e:
        print(f"Error in Johansen Test for Store {store}: {e}")


In [ ]:
for store in selected_stores:
    store_data = df[(df.Store == store) & (df.Open == 1)].copy()
    johansen_test(store_data, store)

## 10. Johansen Cointegration Test

In [ ]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def johansen_test(df_subset, store):
    """
    Apply Johansen Test on multiple series (Sales + Promo2Active).
    """
    df_subset = df_subset.set_index("Date")[["Sales", "Promo2Active"]].asfreq("D").fillna(method="ffill")
    coint_result = coint_johansen(df_subset, det_order=0, k_ar_diff=1)

    print(f"\n Johansen Test - Store {store}")
    trace_stat = coint_result.lr1
    critical_values = coint_result.cvt[:, 1]  # 5% significance
    for i, (stat, crit) in enumerate(zip(trace_stat, critical_values)):
        print(f"Rank {i}: Trace Stat = {stat:.2f}, Crit Value (5%) = {crit}")
        if stat > crit:
            print(" Cointegration Exists at Rank", i)
        else:
            print(" No Cointegration at Rank", i)

# Run Johansen test per store
for store in selected_stores:
    print(f"\n Store {store}")
    store_data = df[(df.Store == store) & (df.Open == 1)].copy()
    johansen_test(store_data, store)

### Johansen Test

- Required when building VAR/VARMAX models on multivariate non-stationary series.
- Determines if variables (like Sales and Promo2Active) are cointegrated.

## 11. Model Building & Forecasting Loop (Per Store)

In [ ]:
# Build VAR/VARMAX model, forecast, and evaluate per store
forecast_results = []

for store in selected_stores:
    print(f"\n Building Model for Store {store}")
    
    #  Get train and test data
    store_data = df[(df.Store == store) & (df.Open == 1)].copy()
    store_data.set_index("Date", inplace=True)
    
    train = store_data[store_data["is_test"] == 0]
    test = store_data[store_data["is_test"] == 1]
    
    #  Use Sales and Promo2Active for modeling
    train_ts = train[["Sales", "Promo2Active"]].asfreq("D").fillna(method="ffill")
    test_ts = test[["Sales", "Promo2Active"]].asfreq("D").fillna(method="ffill")
    
    #  Fit VARMAX Model
    try:
        model = VARMAX(train_ts, order=(2, 0))
        model_fitted = model.fit(disp=False)
    except Exception as e:
        print(f" VARMAX failed for Store {store}: {e}")
        continue

    #  Forecast
    forecast = model_fitted.forecast(steps=len(test_ts))
    forecast.index = test_ts.index
    forecast.columns = ["Sales_forecast", "Promo2Active_forecast"]

    #  Evaluate MAPE
    mape = mean_absolute_percentage_error(test_ts["Sales"], forecast["Sales_forecast"])
    print(f" MAPE for Store {store}: {round(mape * 100, 2)}%")
    
    #  Plot Actual vs Forecasted
    plt.figure(figsize=(10, 4))
    plt.plot(test_ts.index, test_ts["Sales"], label="Actual", color='blue')
    plt.plot(test_ts.index, forecast["Sales_forecast"], label="Forecast", color='orange')
    plt.title(f" Forecast vs Actual - Store {store}")
    plt.xlabel("Date")
    plt.ylabel("Sales")
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()

    #  Save results
    forecast_results.append({
        "Store": store,
        "MAPE": round(mape * 100, 2)
    })

## 12. Final Evaluation Summary

In [ ]:
# Create summary DataFrame

result_df = pd.DataFrame(forecast_results)
result_df.sort_values("MAPE")